Python version of https://github.com/dflynn-volpe/Interpolate_Weather/

Steps:

- Import necessary libraries and set up temporary directory to hold outputs
- Read in data from repository
- Project weather station data and grid shapefile and plot 
- Make raster layer, loop over weather data by day to create interpolated weather layers
- Extract values to grid cells and output data frame of gridded, daily weather values
- Plot raster layer and plot output gridded layers


In [12]:
import geopandas as gpd
import pandas as pd
import os
import requests, zipfile, io

# Set up temporary directory to hold outputs
home = os.path.expanduser("~")
temppath = os.path.join(home, "Interpolate_Weather_Temp_py")

#path = os.getcwd()  
#print ("The current working directory is %s" % path) 

if not os.path.exists(temppath):
    print ("Creating %s" % temppath) 
    os.makedirs(temppath)

In [13]:
wx = pd.read_csv('https://github.com/dflynn-volpe/Interpolate_Weather_Volpe/blob/master/TN_Station_Data_2017-04-01_to_2018-03-31.csv?raw=true')

wx.head()

r = requests.get('https://github.com/dflynn-volpe/Interpolate_Weather_Volpe/blob/master/TN_01dd_fishnet.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

#zipget = httr::GET()
#writeBin(httr::content(zipget, "raw"), 'TN_01dd_fishnet.zip')
